In [16]:
import cropvalidation 
import os
import numpy as np

### TOGO 


Load data

load in togo data as cropmasks

In [17]:
togo_data = []
dir = 'data/TogoBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/TogoBinaryMaps\AgreementMap_Togo_Binary.tif":

        continue
    togo_data.append(cropvalidation.CropMask(f))

togo_labels = [togo_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(togo_data))]

TOGO_BINARY = dict(zip(togo_labels, togo_data))
TOGO_BINARY.get('ASAP_TOGO')

Matched Sets

In [18]:
TOGO_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/togo_reprojected.csv', projection='epsg:32631')
TOGO_VALIDATION.validation = TOGO_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

TOGO_MATCHED = []
for binarymap in np.arange(len(togo_data)):
    TOGO_MATCHED.append(cropvalidation.MatchedSet(togo_data[binarymap], TOGO_VALIDATION))

In [19]:
TOGO_F1_SCORES = []
for binarymap in np.arange(len(TOGO_MATCHED)):
    TOGO_MATCHED[binarymap].remove_filler()
    TOGO_F1_SCORES.append(TOGO_MATCHED[binarymap].get_f1())

TOGO_F1_SCORES

[0.585551330798479,
 0.641860465116279,
 0.40229885057471265,
 0.5622119815668203,
 0.7079646017699115,
 0.19444444444444445,
 0.6218487394957983,
 0.8294573643410852]

KENYA DATA

In [20]:
kenya_data = []
dir = 'data/KenyaBinaryMaps'
for tif in os.listdir(dir):
    f = os.path.join(dir, tif)
    if f == "data/KenyaBinaryMaps\Agreement_Kenya_Binary.tif":
        continue
    kenya_data.append(cropvalidation.CropMask(f))

kenya_labels = [kenya_data[x].getName().split("\\")[-1][:-11] for x in np.arange(len(kenya_data))]

KENYA_BINARY = np.array((kenya_labels, togo_data))

KENYA_BINARY

array([['Asap_Kenya', 'CombinedMap_Kenya', 'Copernicus_Kenya',
        'DE_Kenya', 'EsaWorldcover_Kenya', 'ESRI_Kenya', 'GFSAD_Kenya',
        'Harvest_Kenya'],
      dtype=object)

In [21]:
import rasterio as ras
data = ras.open('data/KenyaBinaryMaps/Asap_Kenya_Binary.tif')
data.crs

CRS.from_epsg(32736)

In [22]:
KENYA_VALIDATION = cropvalidation.ValidationSet(path = 'data/HarvestData/kenya.csv', projection='epsg:4326')
KENYA_VALIDATION.reproject_validation(outProjection='epsg:32736')

In [23]:
KENYA_VALIDATION.validation = KENYA_VALIDATION.validation[['lat', 'lon', 'crop_probability']]

In [24]:
KENYA_MATCHED = []
for binarymap in np.arange(len(kenya_data)):
    KENYA_MATCHED.append(cropvalidation.MatchedSet(kenya_data[binarymap], KENYA_VALIDATION))

In [25]:
KENYA_F1_SCORES = []
for binarymap in np.arange(len(KENYA_MATCHED)):
    KENYA_MATCHED[binarymap].remove_filler()
    KENYA_F1_SCORES.append(KENYA_MATCHED[binarymap].get_f1())

KENYA_F1_SCORES

[0.9485825684347332,
 0.9542852260104248,
 0.19145713420175706,
 0.9237343852728468,
 0.3357440890125174,
 0.6999145664246049,
 0.9562672736140465,
 0.9624882508758438]

Scores to add
- user?
- cropland evaluation & non cropland evaluation